# Installing packages

# Importing Libraries

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import seed
from numpy.random import randn
import numpy as np
from scipy import stats
from math import sqrt
from numpy.random import seed
from numpy.random import randn
from numpy import mean
from scipy.stats import sem
from scipy.stats import t
from termcolor import colored

In [2]:
# function for calculating the t-test for two independent samples
def independent_ttest(data1, data2, alpha):
 # calculate means
 mean1, mean2 = mean(data1), mean(data2)
 # calculate standard errors
 se1, se2 = sem(data1), sem(data2)
 # standard error on the difference between the samples
 sed = sqrt(se1**2.0 + se2**2.0)
 # calculate the t statistic
 t_stat = (mean1 - mean2) / sed
 # degrees of freedom
 df = len(data1) + len(data2) - 2
 # calculate the critical value
 cv = t.ppf(1.0 - alpha, df)
 # calculate the p-value
 p = (1.0 - t.cdf(abs(t_stat), df)) * 2.0
 # return everything
 return t_stat, df, cv, p

## Toy Samples

In [3]:
mu2, sigma2 = 25, 5 # mean and standard deviation
s2 = np.random.normal(mu2, sigma2, 5000)
mu, sigma = 30, 5 # mean and standard deviation
s = np.random.normal(mu, sigma, 5000)
zz = [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]

# Population

In [ ]:
df = pd.read_excel('carcass_2025_may.xlsx')
df['Group'] = df['Amostra'].apply(lambda x: x[0])
df['Day'] = df['Amostra'].apply(lambda x: int(x[1]) if x[1] != 'u' else 0)
df.head()

In [5]:
df.columns = df.columns.str.strip()

In [ ]:
df[df['VG'].isna()]

In [7]:
# df[df['Amostra'] == 'Pu-1']['VG'].mean() 

In [8]:
# df[df['Amostra'] == 'Pu-1']['VG'] = df[df['Amostra'] == 'Pu-1']['VG'].mean() 

In [ ]:
df.describe()

In [10]:
df_backup = df.copy()

In [ ]:
df.head()

In [12]:
df = df.groupby(['Amostra', 'Group', 'Day']).mean().reset_index()

In [ ]:
df

Each distribution is a total of eggs layed during 10 days for each female individual

In [ ]:
df['DCT_VG'] =  df['VG'] - df['RPL32']
df['DCT_VGR'] =  df['VGR'] - df['RPL32']
df.head()

In [ ]:
df['DDCT_VG'] =  df['DCT_VG'] - df['DCT_VG'].min() 
df['DDCT_VGR'] =  df['DCT_VGR'] - df['DCT_VGR'].min()
df.head()

In [ ]:
df['EXP_DDCT_VG'] =  df['DDCT_VG'].apply(lambda x: 2**(-x)) 
df['EXP_DDCT_VGR'] =  df['DDCT_VGR'].apply(lambda x: 2**(-x)) 
df.head()

## Sanity Test

In [17]:
def interpret_ttest(a, b, alpha=0.05):
    t_stat, df, cv, p = independent_ttest(a, b, alpha)
    print('t=%.3f, df=%d, cv=%.5f, p=%.8f' % (t_stat, df, cv, p))
#     # interpret via critical value
#     if abs(t_stat) <= cv:
#         print('Accept null hypothesis (Same distributions).')
#     else:
#         print('Reject the null hypothesis (Different distributions).')
    # interpret via p-value
    if p > alpha:
        print(colored('Accept null hypothesis (Same distributions).', 'green'))
    else:
        print(colored('Reject the null hypothesis (Different distributions).', 'red'))

In [ ]:
interpret_ttest(s,s2)

In [19]:
def interpret_kwtest(a, b, alpha = 0.05):
    stat, p = stats.kruskal(a, b)
    print('Statistics=%.3f, p=%.8f' % (stat, p))
    # interpret

    if p > alpha:
        print(colored('Same distributions (fail to reject H0)', 'green'))
    else:
        print(colored('Different distributions (reject H0)', 'red'))

In [ ]:
interpret_kwtest(s,s2)

In [21]:
from scipy.stats import shapiro
# normality test
# interpret results
def interpret_normaltest(data, alpha=0.05):
    stat, p = shapiro(data)
    print('Statistics=%.3f, p=%.3f' % (stat, p))

    if p > alpha:
        print(colored('Sample looks Gaussian (fail to reject H0)', 'green'))
    else:
        print(colored('Sample does not look Gaussian (reject H0)', 'red'))



In [ ]:
interpret_normaltest(s)

In [ ]:
interpret_normaltest(s2)

In [ ]:
interpret_normaltest(zz)

## Normality Tests

In [ ]:
df.head()

In [ ]:
groups = df.Group.unique()
groups

In [27]:
distributions_vg  = {g: df[df['Group'] == g]['EXP_DDCT_VG'].values for g in groups}
distributions_vgr  = {g: df[df['Group'] == g]['EXP_DDCT_VGR'].values for g in groups}

## Normality Test for VG 

In [ ]:
df

In [29]:
color_map = {
    'P': 'black', 
    'A': 'Green', 
    'B': 'Blue', 
    'C': 'Red', 
    'D': 'Yellow'}

In [ ]:
df_day_stats = df.groupby(['Group', 'Day']).describe().reset_index()
df_day_stats.columns = ['_'.join(tuple(map(str, t))) for t in df_day_stats.columns.values]
df_day_stats['EXP_DDCT_VG_error'] = df_day_stats['EXP_DDCT_VG_std'] / (df_day_stats['EXP_DDCT_VG_count']**(1/2))
df_day_stats['EXP_DDCT_VGR_error'] = df_day_stats['EXP_DDCT_VGR_std'] / (df_day_stats['EXP_DDCT_VGR_count']**(1/2))
df_day_stats['GroupDay'] = df_day_stats['Group_'].astype(str) + df_day_stats['Day_'].astype(str)
df_day_stats['Color'] = df_day_stats['Group_'].map(lambda x: color_map[x])

df_day_stats.to_excel('carcass_stats.xlsx')
df_day_stats[['GroupDay',  'EXP_DDCT_VG_error', 'EXP_DDCT_VGR_error', 'EXP_DDCT_VGR_mean', 'EXP_DDCT_VG_mean', 'EXP_DDCT_VGR_std', 'EXP_DDCT_VGR_max', 'EXP_DDCT_VGR_50%']]


In [ ]:
df_day_stats

In [ ]:
import matplotlib.pyplot as plt 
width = 16
height = 9
plt.figure(figsize=(width, height))
plt.title('VG Ovário')
plt.xlabel('Grupos e Dias')
plt.ylabel('2 ^ (-ddct)')
target_col = 'EXP_DDCT_VG'
for d in [0, 1, 4, 8]: 
    df_plot = df_day_stats[df_day_stats.Day_ == d]
    plt.bar(df_plot['GroupDay'].values, df_plot[f'{target_col}_mean'].values, color=df_plot['Color'].values)
    plt.errorbar(df_plot['GroupDay'].values, df_plot[f'{target_col}_mean'].values, df_plot[f'{target_col}_error'].values, fmt='.', 
                 color='Black', elinewidth=2,capthick=10,errorevery=1, alpha=0.5, ms=4, capsize = 2)
    
plt.savefig('figure.png', dpi=400, transparent=True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt 
width = 16
height = 9
plt.figure(figsize=(width, height))
plt.title('VGR Ovário')
plt.xlabel('Grupos e Dias')
plt.ylabel('2 ^ (-ddct)')
target_col = 'EXP_DDCT_VGR'
for d in [0, 1, 4, 8]: 
    df_plot = df_day_stats[df_day_stats.Day_ == d]
    plt.bar(df_plot['GroupDay'].values, df_plot[f'{target_col}_mean'].values, color=df_plot['Color'].values)
    plt.errorbar(df_plot['GroupDay'].values, df_plot[f'{target_col}_mean'].values, df_plot[f'{target_col}_error'].values, fmt='.', 
                 color='Black', elinewidth=2,capthick=10,errorevery=1, alpha=0.5, ms=4, capsize = 2)
    
plt.savefig('figure.png', dpi=400, transparent=True)
plt.show()

In [ ]:
import seaborn as sns
sns.boxplot(data=df, x="Day", y="EXP_DDCT_VG", hue="Group", palette=color_map)


In [ ]:
import seaborn as sns
sns.boxplot(data=df, x="Day", y="EXP_DDCT_VGR", hue="Group", palette=color_map)


In [ ]:
import seaborn as sns
plt.figure(figsize=(width, height))
plt.title('VG Carcass')
plt.xlabel('Grupos e Dias')
plt.ylabel('2 ^ (-ddct)')
sns.boxplot(data=df, x="Day", y="EXP_DDCT_VG", hue="Group", palette=color_map)
plt.savefig('vg_carcaca.png')
plt.savefig('vg_carcaca.pdf')

In [ ]:
import seaborn as sns
plt.figure(figsize=(width, height))
plt.title('VGR Carcass')
plt.xlabel('Grupos e Dias')
plt.ylabel('2 ^ (-ddct)')
sns.boxplot(data=df, x="Day", y="EXP_DDCT_VGR", hue="Group", palette=color_map)
plt.savefig('vgr_carcaca.pdf')

In [38]:
df.to_excel('carcass_exp_ddct.xlsx')

In [ ]:
plt.rcParams.update({'figure.figsize':(7,5), 'figure.dpi':100})
for k, v in distributions_vg.items():
    print(f'Distributions {k}')
    interpret_normaltest(v)
#     plt.hist(v, bins=7)
#     plt.gca().set(ylabel='Frequency')
#     plt.title(k)
#     plt.show()
    print('\n\n')

## Normality Test for VGR


In [ ]:
plt.rcParams.update({'figure.figsize':(7,5), 'figure.dpi':100})
for k, v in distributions_vgr.items():
    print(f'Distributions {k}')
    interpret_normaltest(v)
    plt.hist(v, bins=10)
    plt.gca().set(ylabel='Frequency')
    plt.title(k)
    plt.show()
    print('\n\n')

## Comparison Scenarios

In [41]:
scenarios_vgr_inter_group = {
'A-B': (distributions_vgr['A'], distributions_vgr['B']), 
'A-C': (distributions_vgr['A'], distributions_vgr['C']), 
'A-D': (distributions_vgr['A'], distributions_vgr['D']), 
'B-C': (distributions_vgr['B'], distributions_vgr['C']), 
'B-D': (distributions_vgr['B'], distributions_vgr['D']), 
'C-D': (distributions_vgr['C'], distributions_vgr['D']), 
}

In [42]:
intra_group_days = [(1,4) ,
(1,8),
(4,8)]


In [43]:
scenarios_vgr_intra_groups = { f'{g}-{d[0]}-{d[1]}': 
                             (df[(df['Group'] == g) & (df['Day'] == d[0])]['VGR'].values, 
                              df[(df['Group'] == g) & (df['Day'] == d[1])]['VGR'].values)
                              for g in distributions_vgr.keys() for d in intra_group_days}

In [44]:
scenarios_vg_inter_group = {
'A-B': (distributions_vg['A'], distributions_vg['B']), 
'A-C': (distributions_vg['A'], distributions_vg['C']), 
'A-D': (distributions_vg['A'], distributions_vg['D']), 
'B-C': (distributions_vg['B'], distributions_vg['C']), 
'B-D': (distributions_vg['B'], distributions_vg['D']), 
'C-D': (distributions_vg['C'], distributions_vg['D']), 
}

In [45]:
scenarios_vg_intra_groups = { f'{g}-{d[0]}-{d[1]}': 
                             (df[(df['Group'] == g) & (df['Day'] == d[0])]['VG'].values, 
                              df[(df['Group'] == g) & (df['Day'] == d[1])]['VG'].values)
                              for g in distributions_vgr.keys() for d in intra_group_days}

In [ ]:
scenarios_vg_inter_group['A-B']

In [ ]:
scenarios_vg_intra_groups['A-1-4']

In [ ]:
scenarios_vgr_inter_group['A-B']

In [ ]:
scenarios_vgr_intra_groups['A-1-4']

### Statistical Tests VG


#### Inter Group VG

In [ ]:
for k, v in scenarios_vg_inter_group.items():
    a, b = v
    print(f'Scenario {k}')
    interpret_kwtest(a,b)
    print('\n\n')

#### Intra Group VG

In [ ]:
for k, v in scenarios_vg_intra_groups.items():
    a, b = v
    print(f'Scenario {k}')
    interpret_kwtest(a,b)
    print('\n\n')

### Statistical Tests VGR

#### Inter Group VGR

In [ ]:
for k, v in scenarios_vgr_inter_group.items():
    a, b = v
    print(f'Scenario {k}')
    interpret_kwtest(a,b)
    print('\n\n')

#### Intra Group VGR

In [ ]:
for k, v in scenarios_vgr_intra_groups.items():
    a, b = v
    print(f'Scenario {k}')
    interpret_kwtest(a,b)
    print('\n\n')